<a href="https://colab.research.google.com/github/iamRahulB/Finetuned-English-to-French-language-translation-using-transformers/blob/main/working_finetune_own_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'language-translation-englishfrench:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F592212%2F1067156%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240222%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240222T114749Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D27441f6bf4c0d345a4ad8c85825f22c79d6782a926990630e88d96b18121b7f14dd947bdcbea650408c76f48644af349e9e9c805351fd0ca61775243421376e413380a89bd62c246e9e13409c3fc53c6c9600ae8ee8b0442dbb498f979fc19235e438922c394ab2ad42581c39ffaf7121207b1273a91d9d2cdfdff5fd7f5b83b110e935a6c41b7816148acba90e11574ebfeb467ed56547666daf4032fa080bd485d54fe83eb3c185e32bc0f543710576e3cbeba19e9c646624b188303ecc4933bd1560aa2876bf65401fc0a9d2b22ef774100de6954947048cd9f1f58ec902cd723f9b64f6f89b32df1856b5de7ea3f5d623f5e36cdc8ee2f55186945857ddf'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 3682883 bytes downloaded
Downloaded and uncompressed: language-translation-englishfrench
Data source import complete.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("/kaggle/input/language-translation-englishfrench/eng_-french.csv")

In [ ]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset

In [ ]:
from datasets import load_dataset


data=load_dataset("csv",data_files={"train":"/kaggle/input/language-translation-englishfrench/eng_-french.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['English words/sentences', 'French words/sentences'],
        num_rows: 175621
    })
})

In [ ]:
prefix="Translate English to French"

In [ ]:
print(data["train"]["English words/sentences"][0:4])

['Hi.', 'Run!', 'Run!', 'Who?']


In [ ]:
checkpoint="Helsinki-NLP/opus-mt-en-ROMANCE"
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
model=TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/313M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-ROMANCE.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/779k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/799k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
source_lang="English words/sentences"
target_lang="French words/sentences"

In [ ]:
# tokenization


def token(example):
  inputs=[prefix + ex for ex in example["English words/sentences"]]
  targets=[var for var in example["French words/sentences"]]
  model_inputs=tokenizer(inputs,truncation=True,max_length=128)
  with tokenizer.as_target_tokenizer():
    labels=tokenizer(targets, truncation=True,max_length=128)
  model_inputs["labels"]=labels["input_ids"]
  return model_inputs

In [ ]:
tokened_data=data.map(token,batched=True)

Map:   0%|          | 0/175621 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
tokened_data

DatasetDict({
    train: Dataset({
        features: ['English words/sentences', 'French words/sentences', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 175621
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

In [ ]:
data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model,return_tensors="np")

In [ ]:
train_dataset=model.prepare_tf_dataset(
    tokened_data["train"],
    batch_size=32,
    shuffle=True,
    collate_fn=data_collator
)

In [ ]:
from transformers import AdamWeightDecay
import tensorflow as tf
optimizer=AdamWeightDecay(learning_rate=2e-5,weight_decay_rate=0.01,)

model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_dataset,epochs=1)

5488/5488 [==============================] - 1488s 264ms/step - loss: 0.5041


In [ ]:
model.save("lang_translation_finetuned")

In [ ]:
model.save_pretrained("finetuned")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


In [ ]:
new=TFAutoModelForSeq2SeqLM.from_pretrained("finetuned")

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
tokenizer.save_pretrained("finetuned_tokenizer")

('finetuned_tokenizer/tokenizer_config.json',
 'finetuned_tokenizer/special_tokens_map.json',
 'finetuned_tokenizer/vocab.json',
 'finetuned_tokenizer/source.spm',
 'finetuned_tokenizer/target.spm',
 'finetuned_tokenizer/added_tokens.json')

In [ ]:
token=AutoTokenizer.from_pretrained("finetuned_tokenizer")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
text="are you ok"

In [ ]:
tokened=token(text,return_tensors="np")

In [ ]:
tokened

{'input_ids': array([[   63,    33, 10563,     0]]), 'attention_mask': array([[1, 1, 1, 1]])}

In [ ]:
out=new.generate(**tokened,max_length=128)

In [ ]:
!pip install -q dl-translate

In [ ]:
import dl_translate as dlt

mt = dlt.TranslationModel()

text_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"
mt.translate(text_hi, source=dlt.lang.HINDI, target=dlt.lang.ENGLISH)

tokenizer_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

'UN chief says there is no military solution in Syria'

In [ ]:
%%capture

from txtai.pipeline import Translation

# Create translation model
translate = Translation()

ModuleNotFoundError: No module named 'txtai'

In [ ]:
# translator = Translator()
translation = translate("This is a test translation into Spanish", "es")
translation

In [ ]:
translator=Translator()

In [ ]:
def translate_text(texts):

   translated_text = translator.translate(texts)
   return translated_text
# Example English text
# Translate text to French


In [ ]:
translate_text("hi")

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
with token.as_target_tokenizer():
  model_text=token.decode(out[0],skip_special_tokens=True)

print(translate_text(model_text))
print(model_text)

AttributeError: 'NoneType' object has no attribute 'group'

English: Hello, how are you?
French: Bonjour, comment allez-vous ?
